# Image Classification using Convolutional Neural Network (CNN) with Keras

## Outline

* [Loading Images](#Loading-Images)
* [Building a Convolutional NN Model](#Building-a-Convolutional-NN-Model)
* [Building a (more complex) Convolutional NN Model](#Building-a-(more-complex)-Convolutional-NN-Model)
* [Model Persistence](#Model-Persistence)

In [ ]:
!pip install opencv-python imutils scikit-learn keras tensorflow

In [ ]:
import os
import random

import cv2
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np

## Loading Images

In [ ]:
%matplotlib inline

In [ ]:
image_paths = list(paths.list_images('data/animals'))

In [ ]:
random.seed(42)
random.shuffle(image_paths)

In [ ]:
image = cv2.imread(image_paths[2500])

plt.figure(figsize=(10, 10))
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_image);

In [ ]:
data = []
labels = []

**Note:** Machine learning models take a *fixed size input*.

In [ ]:
from keras.preprocessing.image import img_to_array

In [ ]:
for image_path in image_paths:
    image = cv2.imread(image_path)
    label = image_path.split(os.path.sep)[-2]
    image = cv2.resize(image, (32, 32), interpolation=cv2.INTER_AREA)
    image = img_to_array(image, data_format='channels_first')
    data.append(image)
    labels.append(label)

In [ ]:
data = np.array(data)
labels = np.array(labels)

Normalize images to the range [0, 1].

In [ ]:
data = data.astype('float') / 255.0

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Flatten
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
lb = LabelBinarizer()

In [ ]:
labels = lb.fit_transform(labels)

In [ ]:
labels

In [ ]:
lb.classes_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=30)

In [ ]:
X_test.shape

## Building a Convolutional NN Model

In [ ]:
# Keras uses "channels first".
input_shape = (3, 32, 32)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [ ]:
H = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_split=0.2,
    batch_size=128
)

In [ ]:
N = np.arange(0, 20)
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(N, H.history['loss'], label='train_loss')
plt.plot(N, H.history['val_loss'], label='val_loss')
plt.title('Training Loss')
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(N, H.history['acc'], label="train_acc")
plt.plot(N, H.history['val_acc'], label='val_acc')
plt.title('Training Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_
))

## Building a (more-complex) Convolutional NN Model

(CONV => RELU) * 3 => POOL => FC

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_split=0.2,
    batch_size=128
)

In [ ]:
N = np.arange(0, 20)
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(N, H.history['loss'], label='train_loss')
plt.plot(N, H.history['val_loss'], label='val_loss')
plt.title('Training Loss')
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(N, H.history['acc'], label="train_acc")
plt.plot(N, H.history['val_acc'], label='val_acc')
plt.title('Training Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_
))

## Model Persistence

In [ ]:
import pickle

In [ ]:
f = open('output/label-bins.pkl', 'wb')
pickle.dump(lb, f)
f.close()

In [ ]:
model.save('output/cnn.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('output/cnn.h5')

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_
))